# Using LLaMA pretrained models for text generation

In this notebook, we will explore how to use pretrained LLaMA models for text generation using the Hugging Face `transformers` library. We will go through the process of loading two different LLaMA models (Llama-2-7b-chat and Meta-Llama-3-8B-Instruct), setting up quantization to optimize memory usage, and performing text generation tasks without any fine-tuning or additional training.

LLaMA (Large Language Model Meta AI) is a family of large language models developed by Meta. These models are trained using a next-token prediction objective, where the model learns to predict the most likely next token in a sequence, enabling coherent and contextually accurate text generation. LLaMA uses pretraining on massive text datasets with a dense transformer architecture, offering competitive results in tasks like summarization, question answering, and text generation. One of the distinguishing features of the LLaMA models is that they are open-source.

In [ ]:
#!pip install transformers accelerate bitsandbytes

In [2]:
## Importing necessary libraries
from transformers import AutoModelForCausalLM, AutoTokenizer
from transformers import BitsAndBytesConfig
import torch
import os

- `AutoModelForCausalLM`: This class is used to load a pre-trained model specifically designed for causal language modeling (i.e., models that predict the next token in a sequence). We will use it to load the Llama models.
- `AutoTokenizer`: Tokenizer is responsible for converting input text into the format that the model can understand (tokens). We use `AutoTokenizer` to automatically fetch the appropriate tokenizer for the model.
- `BitsAndBytesConfig`: This is used to configure model quantization. Quantization is the process of reducing the precision of the model weights to save memory and increase inference speed. We will be using 4-bit quantization for efficiency.

##### Authentication token
To access some Hugging Face pre-trained models, especially models like Llama, we need authentication token. We will retrieve the Hugging Face API token stored in an environment variable `HF_TOKEN`.

In [3]:
access_token = os.getenv('HF_TOKEN')

##### Quantization configuration
We will define a configuration for quantizing the model. Quantization reduces the precision of the model's weights, making it more memory-efficient without significantly sacrificing performance.

In [4]:
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16)

- **`load_in_4bit=True`**: This tells the model to load with 4-bit precision, reducing memory usage significantly.
- **`bnb_4bit_use_double_quant=True`**: This option allows for further compression, improving the efficiency of the quantization.
- **`bnb_4bit_quant_type="nf4"`**: Specifies the type of 4-bit quantization. "NF4" is a specific format designed for better performance.
- **`bnb_4bit_compute_dtype=torch.bfloat16`**: Sets the computation data type to `bfloat16` for faster operations on supported hardware like GPUs.

## Llama-2 model (7B parameters)
LLaMA-2 is a family of LLMs ranging from 7 billion to 70 billion parameters. The 7B parameter model is fine-tuned for conversational use cases, providing robust performance in tasks such as text completion, sentiment analysis, and question-answering. The LLaMA-2 7B Chat model is specifically designed for chat-based tasks, such as generating conversational responses.

- **Tokenizer in LLaMA 2 models**: LLaMA 2 models use SentencePiece as their tokenizer, which is based on subword units. Specifically, the tokenizer is trained using Byte-Pair Encoding (BPE) or a similar subword segmentation approach tailored for the language modeling tasks. This subword-level tokenization enable the model to handle a diverse vocabulary efficiently while reducing the overall number of tokens needed for training. Specifically, the LLaMA tokenizer operates on Unicode byte-level inputs (encodes text at the byte level) and applies pre-tokenization to convert text into a sequence of subwords, optimized for the training corpus. This design allows for better handling of rare words, multilingual text, and arbitrary sequences of characters, making it robust for a wide range of NLP tasks.
- **Context window**: The Llama 2 models has a context window of 4,096 tokens.

In [5]:
model_name = "meta-llama/Llama-2-7b-chat-hf"
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto",
    quantization_config=quantization_config,
    use_auth_token=access_token
)

tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True, token=access_token)

config.json:   0%|          | 0.00/614 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/188 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.62k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

- **Model loading**: We use `AutoModelForCausalLM.from_pretrained()` to load the **LLaMA-2 7B Chat** model, specifying the model name, device configuration (`device_map="auto"` for automatic GPU assignment, if available), and applying quantization for memory efficiency.
- **Tokenizer**: The tokenizer is loaded with `AutoTokenizer.from_pretrained()`, ensuring that it matches the model and can efficiently encode/decode text using the fast tokenizer (`use_fast=True`). The "fast" version of the tokenizer is implemented in Rust for better performance. The tokenizer is responsible for converting human-readable text into a format that the model can understand (tokens), and then converting the model's output back into human-readable text.

##### Generating text with the model

In [6]:
# Define the prompt
prompt = "What is artificial intelligence?"
# Tokenize the input
model_inputs = tokenizer(prompt, return_tensors="pt").to("cuda:0")

# Generate the model output
output = model.generate(**model_inputs)

# Decode and print the output
print(tokenizer.decode(output[0], skip_special_tokens=True))

What is artificial intelligence?
 nobody knows.

Artificial intelligence (AI) is a term used to describe a wide range of technologies that enable machines to perform tasks that typically require human intelligence, such as learning, problem-solving, and decision-making. However, despite its widespread use, there is no universally accepted definition of AI.

The term "artificial intelligence" was coined in 1956 by John McCarthy, a computer scientist who was organizing a conference on the topic. At the time, McCarthy defined AI as "the study of how to make machines behave in ways that would be considered intelligent if done by humans." Since then, the definition of AI has evolved to include a broader range of technologies and applications, but the core idea of AI remains the same: enabling machines to perform tasks that require human-like intelligence.

There are many different approaches to building AI systems, and each approach has its own strengths and weaknesses. Some of the most com

- **Tokenize the input**: The prompt is tokenized using the `tokenizer()`, which converts the text into tokens that the model can understand. The `return_tensors="pt"` argument ensures the output is in PyTorch tensor format, and `.to("cuda:0")` moves the tokenized input to the GPU for faster processing.
- **Generate the output**: We use the `model.generate()` method to generate text based on the model’s understanding of the prompt. The `**model_inputs` unpacks the tokenized inputs for the model.
- **Decode the output**: The model's output is a sequence of tokens. We use `tokenizer.decode()` to convert the tokens back into human-readable text, with `skip_special_tokens=True` to remove any special tokens (e.g., padding or end-of-sequence tokens).

## Llama-3 model (8B parameters)
LLaMA-3 is a family of LLMs ranging from 8 billion to 70 billion parameters. The 8B parameter model is fine-tuned for dialogue applications, providing robust performance in tasks such as text completion, sentiment analysis, and question-answering.

- **Tokenizer in LLaMA 3 models**: The tokenizer in Llama 3 models is the PreTrainedTokenizerFast, an optimized version designed to enhance performance while maintaining accuracy. This tokenizer supports:
  - Fast tokenization: Implemented in Rust, it allows for fast tokenization speeds.
  - Unicode handling: It efficiently processes Unicode byte-level inputs, encoding text at the byte level to ensure comprehensive character coverage.
  - Subword segmentation: Utilizes subword units for tokenization, which helps manage a diverse vocabulary with fewer tokens.
- **Context window**: The Llama 3 models has a context window of 8,192 tokens.

In [7]:
model_name = "meta-llama/Meta-Llama-3-8B-Instruct"
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto",
    quantization_config=quantization_config,
    use_auth_token=access_token
)

tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True, token=access_token)

config.json:   0%|          | 0.00/654 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

...

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/187 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/51.0k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/73.0 [00:00<?, ?B/s]

##### Generating text with the model

In [8]:
# Define the prompt
prompt = "What is artificial intelligence?"
# Tokenize the input
model_inputs = tokenizer(prompt, return_tensors="pt").to("cuda:0")

# Generate the model output
output = model.generate(**model_inputs)

# Decode and print the output
print(tokenizer.decode(output[0], skip_special_tokens=True))

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


What is artificial intelligence? Artificial intelligence (AI) is a type of computer science that enables machines to perform tasks that typically require human intelligence, such as learning, problem-solving, and decision-making. AI is a broad field that encompasses various techniques, including machine learning, natural language processing, computer vision, and robotics. AI is often used to analyze data, recognize patterns, and make predictions, which can be used to improve decision-making, automate processes, and enhance user experiences. There are several types of AI, including:
1. Narrow or Weak AI: This type of AI is designed to perform a specific task, such as playing chess or recognizing faces.
2. General or Strong AI: This type of AI is designed to perform any intellectual task that a human can, such as understanding language or making decisions.
3. Superintelligence: This type of AI is significantly more intelligent than humans and has the potential to revolutionize various as

The output from Llama-2-7B-Chat is concise and conversational, providing a balanced summary of artificial intelligence with examples, while Llama-3-8B-Instruct produces a significantly longer response, offering extensive elaboration, lists, and detailed subpoints that go far beyond the initial prompt.